In [22]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from typing import List, Dict, Any, Optional, Tuple, Union

from lib.utils import get_polygon_root, get_nyse_calendar, get_93, get_polygon_key
from lib.fetcher import HttpRequestFetcher, BatchRequestExecutor

In [2]:
base_path = get_polygon_root()
api_key = get_polygon_key()
tickers = get_93()

## Get urls

In [39]:
def get_nyse_date_tups(start: str, end: str = 'today', time_detail=True):
    if end == 'today': end = pd.Timestamp.now().strftime('%Y-%m-%d') # get today! 
    assert pd.Timestamp(start) < pd.Timestamp(end), "start date must be before end date"

    nyse = get_nyse_calendar(start, end) # get nyse calendar

    decode_str = "%Y-%m-%d %H:%M:%S" if time_detail else "%Y-%m-%d" # decode str
    func = lambda x: pd.to_datetime(x, utc=True).tz_convert('America/New_York').strftime(decode_str) # convert to nyse tz
    tups = [(func(a), func(b)) for a, b in zip(nyse['market_open'], nyse['market_close'])] # get tups of open/close, formatted with func
    return tups

def make_urls(tickers: Union[List[str], str], tups: [Tuple[str, str]]):

    def _make_url(ticker, start, end, limit=1000, adjusted=True, api_key=get_polygon_key()):
        base_url = "https://api.polygon.io/v2/aggs/ticker/"
        adj = 'true' if adjusted else 'false'
        url = f"{base_url}{ticker}/range/1/minute/{start}/{end}?adjusted={adj}&sort=asc&limit={limit}&apiKey={api_key}"
        return url
    
    def _validate(tickers: Union[List[str], str]) -> List[str]:
        if isinstance(tickers, str): tickers = [tickers] # make sure tickers is a list
        assert len(tickers) > 0, "list of tickers must be non-empty" # make sure tickers is non-empty
        return tickers
    
    tickers = _validate(tickers)
    urls = [[_make_url(ticker, date1, date2) for date1, date2 in tups] for ticker in tickers]
    
    if len(tickers) == 1: urls = urls[0] # if only one ticker, flatten list to avoid nested list [[data]] -> [data]

    return urls

def estimate_time(urls, batch_size=1000, req_time=1):
    n_urls = sum([len(url) for url in urls])
    total_time_hrs = n_urls / batch_size * req_time/60
    print(f"Estimated time for {n_urls} requests @ {req_time}s per API call: {total_time_hrs:0.2f} hrs")

In [44]:
end = pd.Timestamp.now().strftime("%Y-%m-%d")
start = (pd.Timestamp.now() - pd.DateOffset(months=1)).strftime("%Y-%m-%d")
tickers_ = tickers[:2]
print(f"Getting data for {tickers_} from {start} to {end}")
tups = get_nyse_date_tups(start, end, time_detail=False)
print(len(tups))
urls = make_urls(tickers_, tups)
estimate_time(urls)

Getting data for ['AAPL', 'ACN'] from 2023-09-09 to 2023-10-09
21
Estimated time for 42 requests @ 1s per API call: 0.00 hrs


In [45]:
urls

[['https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2023-09-11/2023-09-11?adjusted=true&sort=asc&limit=1000&apiKey=None',
  'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2023-09-12/2023-09-12?adjusted=true&sort=asc&limit=1000&apiKey=None',
  'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2023-09-13/2023-09-13?adjusted=true&sort=asc&limit=1000&apiKey=None',
  'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2023-09-14/2023-09-14?adjusted=true&sort=asc&limit=1000&apiKey=None',
  'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2023-09-15/2023-09-15?adjusted=true&sort=asc&limit=1000&apiKey=None',
  'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2023-09-18/2023-09-18?adjusted=true&sort=asc&limit=1000&apiKey=None',
  'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2023-09-19/2023-09-19?adjusted=true&sort=asc&limit=1000&apiKey=None',
  'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2023-09-20/2023-09-20?

In [5]:
print(f"len(urls): {len(urls)}")

len(urls): 4


In [100]:
tickers_ = tickers[0]
print(f"Getting data for {len(tickers_)} tickers")

Getting data for 4 tickers


In [72]:
fetcher = HttpRequestFetcher(rps=2)
executor = BatchRequestExecutor()

In [76]:
results = executor.execute(fetcher, urls)

AttributeError: __aenter__